# COGS 108 - Capstone Project

## Project links, files, and basic information

### Websites with datasets:
- San Diego Vehicle Stops:  https://data.sandiego.gov/datasets/police-vehicle-stops/
- Dan Diego Population Data:  http://www.city-data.com/city/San-Diego-California.html

### Websites of needed information:
- San Diego police service areas https://www.sandiego.gov/police/services/divisions (vehcle stop data only records the first two digits)
- San Diego zip code map: http://www.city-data.com/zipmaps/San-Diego-California.html

### Names of datasets
#### *Vehicle Stops*
- 'vehicle_stops_2017.csv'
- 'vehicle_stops_2016.csv'
- 'vehicle_stops_2015.csv'
- 'vehicle_stops_2014.csv'

#### *Vehicle Stops Details*
- 'vehicle_stops_search_details_2017.csv'
- 'vehicle_stops_search_details_2016.csv'
- 'vehicle_stops_search_details_2015.csv'
- 'vehicle_stops_search_details_2014.csv'

#### *Files needed to read Vehicle Stops information*
- Race Codes: 'vehicle_stops_race_codes.csv'    
- Title explanations for Vehicle Stops data: 'vehicle_stops_dictionary.csv'
- Title explanations for Vehicle Stops Details data: 'vehicle_stops_search_details_dictionary.csv'
- Possible actions taken when stopped for Vehicle Stops Details data: 'vehicle_stops_search_details_description_list.csv'

In [2]:
# Imports
%matplotlib inline

# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Web scrapping
import sys
!conda install --yes --prefix {sys.prefix} beautifulsoup4

# Data analysis
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

import requests
import PyPDF2 as pdf
#import urllib2

Solving environment: ...working... done

# All requested packages already installed.



C:\Users\Gordee\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
df_stops = pd.read_csv('vehicle_stops_2017.csv')
df_stops_info = pd.read_csv('vehicle_stops_search_details_2017.csv')

In [ ]:
print(df_stops_info.shape)

In [7]:
import os

currDir = 'zip_pop_data\\'
for currZip in os.listdir('zip_pop_data'):  
    try:
        file = os.fsdecode(currZip)
        fpdf = pdf.PdfFileReader(currDir + file)
        page = fpdf.getPage(0).extractText()

        # Gets the beginning and end of the data we want
        index = page.find('Population\nPercent\nTotal Population')
        indexEnd = page.find('Source: SANDAG, Current Estimates (2010)\nPopulation by Race')
        text = page[index+19:indexEnd-1]
        text = list(text)
        for index, item in enumerate(text):
            if item == "\n":
                text[index] = '/'

        text = ''.join(text)
        text = text.split('/')
        groups = list()
        percentages = list()
        populations = list()
        cols = ['Group', 'Population', 'Percent']

        for item in text:
            if '%' in item:
                percentages.append(item)
            elif item[0].isnumeric():
                populations.append(item)
            else:
                groups.append(item) 


        p_df = pd.DataFrame(columns = cols)
        p_df['Group'] = groups
        p_df['Population'] = populations
        p_df['Percent'] = percentages
        print(file)
        print(p_df)
        print('\n')
    except PermissionError:
        continue

92014.pdf
              Group Population Percent
0  Total Population     13,432    100%
1             White     11,481     85%
2             Asian        797      6%
3          Hispanic        686      5%
4       Two or More        329      2%
5             Black         75      1%
6             Other         46     <1%
7   American Indian         11     <1%
8  Pacific Islander          7     <1%


92025.pdf
              Group Population Percent
0  Total Population     48,275    100%
1          Hispanic     25,714     53%
2             White     18,382     38%
3             Asian      1,986      4%
4             Black        935      2%
5       Two or More        857      2%
6   American Indian        215     <1%
7  Pacific Islander         97     <1%
8             Other         89     <1%


92091.pdf
              Group Population Percent
0  Total Population      1,621    100%
1             White      1,408     87%
2          Hispanic         99      6%
3             Asian         80

92123.pdf
              Group Population Percent
0  Total Population     26,343    100%
1             White     13,560     51%
2          Hispanic      5,170     20%
3             Asian      3,679     14%
4             Black      2,169      8%
5       Two or More      1,337      5%
6  Pacific Islander        193      1%
7   American Indian        162      1%
8             Other         73     <1%


92124.pdf
              Group Population Percent
0  Total Population     30,332    100%
1             White     18,498     61%
2          Hispanic      4,844     16%
3             Asian      3,031     10%
4             Black      1,868      6%
5       Two or More      1,734      6%
6  Pacific Islander        157      1%
7   American Indian        125     <1%
8             Other         75     <1%


92126.pdf
              Group Population Percent
0  Total Population     73,037    100%
1             Asian     31,629     43%
2             White     23,761     33%
3          Hispanic     10,157

PdfReadError: EOF marker not found